In [ ]:
# torch imports
import torch
import torch._dynamo
torch._dynamo.config.suppress_errors = True
# Misc imports
import os 
import sys
import seaborn as sns
from pathlib import Path
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSegDev')
# Ionpy imports
from ionpy.util import Config
from ionpy.analysis import ResultsLoader

# Define some useful paths.
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
       '/storage'
))
# Set some defaults
rs = ResultsLoader()
sns.set_style("darkgrid")
torch.set_printoptions(linewidth=200)

# For using code without restarting.
%load_ext autoreload
%autoreload 2
# For using yaml configs.
%load_ext yamlmagic

In [ ]:
%%yaml default_cfg 

experiment:
    exp_root: '?'
    inference_seed: 40

inference_data:
    split: '?'

log:
    root: '?'
    save_preds: False 
    log_interval: 10 
    log_pixel_stats: False 
    gether_inference_stats: True
    compute_global_metrics: False 

dataloader:
    batch_size: '?' 
    num_workers: '?' 
    pin_memory: True 

In [ ]:
%%yaml calibration_cfg 

local_calibration:
    num_prob_bins: 15
    neighborhood_width: 3

global_calibration:
    num_classes: 1 
    num_prob_bins: 15
    neighborhood_width: 3

In [ ]:
%%yaml model_cfg 

# For standard datasets
#####################################
model:
    pred_label: 0     
    _type: "standard"
    pretrained_exp_root : None
    checkpoint: 'min-val-abs_area_estimation_error'

In [ ]:
%%yaml aug_cfg

# inference_augmentations:
#     independent: True
#     visual:
#         use_mask: False 
#         added_noise_max_sigma: 0.01
#         gamma_scaling_max: 0.1
#         bias_field_probability: 0.5
#         gamma_scaling_probability: 0.5
#         added_noise_probability: 0.5

## Gather Inference Options.

In [ ]:
%%yaml experiment_cfg 
############################################################################################################
## NAMING FIELDS
group: "UVS_MegaMulticontextInferenceAugSweep"

############################################################################################################
## EXPERIMENTAL VARIABLES
base_model: "/storage/vbutoi/scratch/ESE/training/07_18_24_halle-more-datasets/20240718_172310-AMNK-75889380b16faf8d16a467fa2f1cf2f7" # New and improved

# Change this for debugging
dataloader:
    batch_size: 1
    num_workers: 1

experiment:
    support_size: 32
    num_supports: 10 

inference_data:
    _class: "ese.datasets.Segment2D"
    label: 0
    split: "val"
    support_split: "train"
    task: 
        - "ACDC/Challenge2017/MRI/2"
        - "PanDental/v1/XRay/0"
        - "PanDental/v2/XRay/0"
        - "SCD/LAS/MRI/2"
        - "SCD/VIS_human/MRI/2"
        - "SCD/LAF_Post/MRI/2"
        - "SCD/VIS_pig/MRI/2"
        - "SCD/LAF_Pre/MRI/2"
        - "SpineWeb/Dataset7/MR/0"
        - "STARE/retrieved_2021_12_06/Retinal/0"
        - "WBC/CV/EM/0"
        - "WBC/JTSC/EM/0"

support_inference_augmentations:
    independent: True
    spatial:
        affine_probability: 1.0 
        warp_probability: 0.0
        max_translation: 
            - 0.0
            - 5.0
            - 10.0
            - 15.0
            - 20.0
        max_rotation: 
            - 0.0
            - 90.0
            - 180.0
            - 270.0
            - 360.0

In [ ]:
# Local imports
from ese.analysis.analysis_utils.submit_utils import get_ese_inference_configs

# Get the configs for the different runs.
# base_cfg = Config(default_cfg).update([calibration_cfg, model_cfg])
base_cfg = Config(default_cfg).update([calibration_cfg, model_cfg, aug_cfg])

# For inference experiments, we don't add the date by default.
add_date = True
use_best_models = False 
# Get the different experiment cfg yamls.
updated_base_cfg, inf_cfgs = get_ese_inference_configs(
    exp_cfg=experiment_cfg, 
    base_cfg=base_cfg,
    add_date=add_date,
    use_best_models=use_best_models
)

In [ ]:
len(inf_cfgs)

## Running Jobs

In [ ]:
from ese.analysis.run_inference import get_cal_stats

In [ ]:
from ese.experiment import run_ese_exp

###### Run individual jobs
run_ese_exp(
    config=inf_cfgs[0], 
    job_func=get_cal_stats,
    run_name='debug',
    show_examples=True,
    gpu='0',
)

In [ ]:
# from ese.experiment import submit_ese_exps 

# #### Run Batch Jobs
# submit_ese_exps(
#     group="inference",
#     base_cfg=updated_base_cfg,
#     add_date=add_date,
#     exp_cfg=experiment_cfg,
#     config_list=inf_cfgs,
#     job_func=get_cal_stats,
#     available_gpus=['0', '1', '2', '3', '4', '5', '6', '7']
# )